In [22]:
#importing libraries
import pandas as pd

#API
import requests

#webscraping
from bs4 import BeautifulSoup

# Automating
import time
from time import sleep
import datetime
import warnings
import sys

#### In this project, I will try to classify posts from two subreddits: NBA and Soccer.

Before attempting to collect large amount of data for both subreddits, I will be trying to test my code by collecting small amount of data from NBA subreddit.

In [12]:
# Set base url
url = 'https://api.pushshift.io/reddit/search/submission/?subreddit='

# Set params
params = {
    'subreddit': 'NBA',
    'size': 50,
    'lang':True,
    'before':1602186091, # epoch (another version of a timestamp)
}

# Make request
res = requests.get(url, params)
res

<Response [200]>

In [13]:
# Examine object
res.text
res.json()

{'data': [{'all_awardings': [],
   'allow_live_comments': False,
   'author': 'Throwawayhayday69',
   'author_flair_css_class': 'NBA',
   'author_flair_richtext': [{'e': 'text', 't': 'NBA'}],
   'author_flair_template_id': 'e5aa3fb6-3feb-11e8-8409-0ef728aaae7a',
   'author_flair_text': 'NBA',
   'author_flair_text_color': 'dark',
   'author_flair_type': 'richtext',
   'author_fullname': 't2_4uh4u84p',
   'author_patreon_flair': False,
   'author_premium': False,
   'awarders': [],
   'can_mod_post': False,
   'contest_mode': False,
   'created_utc': 1602185884,
   'domain': 'self.nba',
   'full_link': 'https://www.reddit.com/r/nba/comments/j7jviq/any_other_lebron_fans_luke_warm_on_lebron_going/',
   'gildings': {},
   'id': 'j7jviq',
   'is_crosspostable': True,
   'is_meta': False,
   'is_original_content': False,
   'is_reddit_media_domain': False,
   'is_robot_indexable': True,
   'is_self': True,
   'is_video': False,
   'link_flair_background_color': '',
   'link_flair_richtext': 

In [16]:
# Extract one post from the res.json() object
res.json()['data'][0]

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'Throwawayhayday69',
 'author_flair_css_class': 'NBA',
 'author_flair_richtext': [{'e': 'text', 't': 'NBA'}],
 'author_flair_template_id': 'e5aa3fb6-3feb-11e8-8409-0ef728aaae7a',
 'author_flair_text': 'NBA',
 'author_flair_text_color': 'dark',
 'author_flair_type': 'richtext',
 'author_fullname': 't2_4uh4u84p',
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1602185884,
 'domain': 'self.nba',
 'full_link': 'https://www.reddit.com/r/nba/comments/j7jviq/any_other_lebron_fans_luke_warm_on_lebron_going/',
 'gildings': {},
 'id': 'j7jviq',
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': True,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',


In [17]:
#let's make a dataframe
nba_df = pd.DataFrame(res.json()['data'])
nba_df.head(3)

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,...,removed_by_category,secure_media,secure_media_embed,thumbnail_height,thumbnail_width,url_overridden_by_dest,link_flair_css_class,link_flair_template_id,link_flair_text,author_flair_background_color
0,[],False,Throwawayhayday69,NBA,"[{'e': 'text', 't': 'NBA'}]",e5aa3fb6-3feb-11e8-8409-0ef728aaae7a,NBA,dark,richtext,t2_4uh4u84p,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,Riven4,Lakers1,"[{'e': 'text', 't': 'Lakers'}]",cc0d49ae-3feb-11e8-8327-0e3e867879aa,Lakers,dark,richtext,t2_hd2tq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,throwawaynashville11,None,[],NaN,None,NaN,text,t2_14j6yk,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We have many unecessary columns, for our classification purposes let's select the columns we're interested in 

In [18]:
# Extract columns
nba_df = nba_df.loc[:, ['title','created_utc', 'selftext','subreddit','author','media_only','permalink']]

#making sure we only data from 1 subreddit
nba_df['subreddit'].nunique()

1

In [23]:
nba_df.head(3)

,title,created_utc,selftext,subreddit,author,media_only,permalink
0,Any other lebron fans luke warm on lebron goin...,1602185884,I've been a fan of lebron for most of my life....,nba,Throwawayhayday69,False,/r/nba/comments/j7jviq/any_other_lebron_fans_l...
1,If the Bucks have the same amount of regular s...,1602185798,Lets say the Bucks win around 62 games and are...,nba,Riven4,False,/r/nba/comments/j7juid/if_the_bucks_have_the_s...
2,name a nba player i’ll drop a bar around it,1602185792,this is copied from an older thread that got p...,nba,throwawaynashville11,False,/r/nba/comments/j7jufm/name_a_nba_player_ill_d...


In [21]:
nba_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        50 non-null     object
 1   created_utc  50 non-null     int64 
 2   selftext     50 non-null     object
 3   subreddit    50 non-null     object
 4   author       50 non-null     object
 5   media_only   50 non-null     bool  
 6   permalink    50 non-null     object
dtypes: bool(1), int64(1), object(5)
memory usage: 2.5+ KB


#### Now that we made sure our code is working we can make a function to automate this process

In [30]:
def get_posts(subreddit, n_iter, epoch_right_now): # subreddit name and number of times function should run
    # store base url variable for pushshift API
    base_url = 'https://api.pushshift.io/reddit/search/submission/?subreddit='

    # Making an empty list to append out dataframes  
    df_list = []
    
    # save current epoch, used to iterate in reverse through time
    # you can get it from here: https://www.epochconverter.com/
    current_time = epoch_right_now
    
    # set up for loop
    for post in range(n_iter):
        
        # instantiate get request
        res = requests.get(
            
            # requests.get takes base_url and params
            base_url,
            
            # parameters for get request
            params = {
                
                # specify subreddit
                'subreddit' : subreddit,
                
                # specify number of posts to pull
                'size' : 100,
                
                # Restrict based on the default language of the subreddit
                'lang' : True,
                
                # pull everything from current time backward
                'before' : current_time }
        )
        
        # take data from most recent request, store as df
        df = pd.DataFrame(res.json()['data'])
        
        # pull specific columns from dataframe for analysis
        df = df.loc[:, ['title',
                        'created_utc', 
                        'selftext',
                        'subreddit',
                        'author',
                        'media_only',
                        'permalink']]
        
        # append to empty dataframe list
        df_list.append(df)
        
        # add wait time
        time.sleep(30)
        
        # set current time counter back to last epoch in recently grabbed df
        current_time = df['created_utc'].min()

    # return one dataframe for all requests
    return pd.concat(df_list, axis=0)

# Adapated from Tim Book's Lesson Example

In [32]:
# nba = get_posts('NBA', 500, 1602187760)

In [34]:
nba.shape

(50000, 7)

In [35]:
nba.head()

,title,created_utc,selftext,subreddit,author,media_only,permalink
0,LeBron James More Proud Of His Legacy 'Off-Cou...,1602187737,,nba,auscrisos,False,/r/nba/comments/j7kgw4/lebron_james_more_proud...
1,[Sports Illustrated] Former Pistons Guard Isia...,1602187137,[Source](https://www.si.com/nba/2020/10/08/isi...,nba,educationprimo,False,/r/nba/comments/j7k9ut/sports_illustrated_form...
2,Who Would Make A Better Wide Receiver: Westbro...,1602187128,Westbrook is built like a wide receiver. He’s ...,nba,sonicoduh1125,False,/r/nba/comments/j7k9rm/who_would_make_a_better...
3,"At the top of their peaks, who are you taking?...",1602186939,"1.) At the top of their **PEAKS,** who are you...",nba,Drakey504,False,/r/nba/comments/j7k7nd/at_the_top_of_their_pea...
4,Is steve nash the worst player with atleast 2 ...,1602186925,Is he? Can you think of a player that has 2 mv...,nba,Throwawayhayday69,False,/r/nba/comments/j7k7hy/is_steve_nash_the_worst...


In [36]:
nba.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 99
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        50000 non-null  object
 1   created_utc  50000 non-null  int64 
 2   selftext     49635 non-null  object
 3   subreddit    50000 non-null  object
 4   author       50000 non-null  object
 5   media_only   50000 non-null  bool  
 6   permalink    50000 non-null  object
dtypes: bool(1), int64(1), object(5)
memory usage: 2.7+ MB


In [37]:
nba.to_csv('./data/nba_post.csv', index=False)

In [38]:
# soccer = get_posts('soccer', 500, 1602187760)

In [39]:
soccer.head()

,title,created_utc,selftext,subreddit,author,media_only,permalink
0,Iceland 2 - [1] Romania - Alexandru Maxim Pena...,1602187710,,soccer,overhungoasis,False,/r/soccer/comments/j7kgkd/iceland_2_1_romania_...
1,Iceland 2-[1] Romania - Alexandru Maxim penalt...,1602187687,,soccer,NotMeladroit,False,/r/soccer/comments/j7kgar/iceland_21_romania_a...
2,Iceland 2 - [1] Romania - Alexandru Maxim Penalty,1602187676,[deleted],soccer,[deleted],False,/r/soccer/comments/j7kg64/iceland_2_1_romania_...
3,[Goff] Ben Olsen and D.C. United have agreed t...,1602187472,,soccer,NittanyOrange,False,/r/soccer/comments/j7kdvo/goff_ben_olsen_and_d...
4,Manchester City ruining English football? Thou...,1602187327,[removed],soccer,abae72,False,/r/soccer/comments/j7kc76/manchester_city_ruin...


In [40]:
soccer.to_csv('./data/soccer_post.csv', index=False)

In [41]:
soccer.shape

(50000, 7)

In [42]:
soccer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 99
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        50000 non-null  object
 1   created_utc  50000 non-null  int64 
 2   selftext     49875 non-null  object
 3   subreddit    50000 non-null  object
 4   author       50000 non-null  object
 5   media_only   50000 non-null  bool  
 6   permalink    50000 non-null  object
dtypes: bool(1), int64(1), object(5)
memory usage: 2.7+ MB


Now that we have two datasets for each subreddit, let's combine them in one dataframe.

In [44]:
both = pd.concat([nba,soccer])

In [45]:
both.shape

(100000, 7)

In [51]:
both.head(3)

,title,created_utc,selftext,subreddit,author,media_only,permalink
0,LeBron James More Proud Of His Legacy 'Off-Cou...,1602187737,,nba,auscrisos,False,/r/nba/comments/j7kgw4/lebron_james_more_proud...
1,[Sports Illustrated] Former Pistons Guard Isia...,1602187137,[Source](https://www.si.com/nba/2020/10/08/isi...,nba,educationprimo,False,/r/nba/comments/j7k9ut/sports_illustrated_form...
2,Who Would Make A Better Wide Receiver: Westbro...,1602187128,Westbrook is built like a wide receiver. He’s ...,nba,sonicoduh1125,False,/r/nba/comments/j7k9rm/who_would_make_a_better...


In [47]:
#dropping duplicate posts
both_dropped_duplicate = both.drop_duplicates(subset='title')

In [48]:
both_dropped_duplicate.shape

(94739, 7)

In [49]:
both_dropped_duplicate['subreddit'].value_counts(normalize=True)

soccer    0.500881
nba       0.499119
Name: subreddit, dtype: float64

In [52]:
# now let's create a label column to serve as our target value to predict.
# for this Target column, will assign nba:0, soccer:1
both_dropped_duplicate['target'] = both_dropped_duplicate['subreddit'].map({'nba':0, 'soccer':1})

<ipython-input-52-123524bf2d4d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  both_dropped_duplicate['target'] = both_dropped_duplicate['subreddit'].map({'nba':0, 'soccer':1})


In [57]:
both_dropped_duplicate.head(2)

,title,created_utc,selftext,subreddit,author,media_only,permalink,target
0,LeBron James More Proud Of His Legacy 'Off-Cou...,1602187737,,nba,auscrisos,False,/r/nba/comments/j7kgw4/lebron_james_more_proud...,0
1,[Sports Illustrated] Former Pistons Guard Isia...,1602187137,[Source](https://www.si.com/nba/2020/10/08/isi...,nba,educationprimo,False,/r/nba/comments/j7k9ut/sports_illustrated_form...,0


In [58]:
both_dropped_duplicate[50_000:].head(2)

,title,created_utc,selftext,subreddit,author,media_only,permalink,target
19,AC Milan's form since June. Undefeated in 21 g...,1601823614,,soccer,dendidendi,False,/r/soccer/comments/j509uz/ac_milans_form_since...,1
20,Apollon Smirnis vs Larisa Soccer Live Stream -...,1601823611,,soccer,goodsports2020,False,/r/soccer/comments/j509tx/apollon_smirnis_vs_l...,1


Great. Now that we have a combined dataset that has both subreddits with a label column, let's shuffle the rows so it's spread across equaly for both subbreddits and then reset the index.

In [64]:
#shuffling df
both_dropped_duplicate = both_dropped_duplicate.sample(frac = 1)

In [65]:
both_dropped_duplicate.head(3)

,title,created_utc,selftext,subreddit,author,media_only,permalink,target
50,If LeBron manage to lose to this Final and end...,1601223996,How will affect his legacy if he somehow were ...,nba,u87hi,False,/r/nba/comments/j0tyu3/if_lebron_manage_to_los...,0
20,[Ramirez]Anthony Davis shot 68.0% from mid-ran...,1598987437,,nba,adonisgawd,False,/r/nba/comments/ikqojx/ramirezanthony_davis_sh...,0
87,"[David Squires] FFS, lads; I’ve only got eight...",1601845822,,soccer,JonSnowAzorAhai,False,/r/soccer/comments/j56nzm/david_squires_ffs_la...,1


In [66]:
both_dropped_duplicate.shape

(94739, 8)

In [67]:
both_dropped_duplicate.reset_index(drop=True)

,title,created_utc,selftext,subreddit,author,media_only,permalink,target
0,If LeBron manage to lose to this Final and end...,1601223996,How will affect his legacy if he somehow were ...,nba,u87hi,False,/r/nba/comments/j0tyu3/if_lebron_manage_to_los...,0
1,[Ramirez]Anthony Davis shot 68.0% from mid-ran...,1598987437,,nba,adonisgawd,False,/r/nba/comments/ikqojx/ramirezanthony_davis_sh...,0
2,"[David Squires] FFS, lads; I’ve only got eight...",1601845822,,soccer,JonSnowAzorAhai,False,/r/soccer/comments/j56nzm/david_squires_ffs_la...,1
3,"Ivor Pandur (20, GK, HNK Rijeka) is close to s...",1598534836,,soccer,jer_ili_jel,False,/r/soccer/comments/ihl04p/ivor_pandur_20_gk_hn...,1
4,This is how strong BAYERN MUNICH is this seaso...,1598200937,,soccer,Terrible-Discount,False,/r/soccer/comments/if6eca/this_is_how_strong_b...,1
...,...,...,...,...,...,...,...,...
94734,"Sir Alex Ferguson to Klopp: ""Your personality ...",1595881071,,soccer,flexiblelion,False,/r/soccer/comments/hyzz1n/sir_alex_ferguson_to...,1
94735,Since we all know Lebron will avg his playoff ...,1597347639,"For me, Lakers bench sucks and their starters ...",nba,kawhiii16,False,/r/nba/comments/i96pu2/since_we_all_know_lebro...,0
94736,MAN UNITED EUROPA LEAGUE EXIT CHAT!!!!,1597692887,[removed],soccer,KINGDEVIL86,False,/r/soccer/comments/ibl4p2/man_united_europa_le...,1
94737,Carragher: 'Liverpool won't go near Messi',1598628326,,soccer,AdamColeIsTheGOAT,False,/r/soccer/comments/ii923t/carragher_liverpool_...,1


Now we're ready to save our final dataset to be used for classification models.

In [68]:
#Saving our final dataset
both_dropped_duplicate.to_csv('./data/combined_nba_soccer.csv', index=False)